In [3]:
valid_amount = []
def _amount_():
    try:
        flag = True
        amount = input()
        for a in amount:
            if a not in str(list(range(0,10))):
                flag = False
                raise TypeError('Amount must be numerical')
        if flag:
            valid_amount.append(float(amount))       
    except TypeError:
        _amount_()

class Account:
    def __init__(self, account_id,balance, interest = 0):
        
        '''checking if account id is string'''

        if not isinstance(account_id,str):
            raise TypeError('Account ID must be string')
        if not isinstance(balance,str):
            raise TypeError('Balance must be string')    
        for id in account_id:
            if id not in str(list(range(0,10))):
                raise ValueError('Account ID must be within 0-9')
        for num in balance:
            if num not in str(list(range(0,10))):
                raise ValueError('Balance must be numerical')
        
        '''assign the attributes'''

        self.account_id = account_id
        self.balance = float(balance)
        self.interest = interest

    '''print message'''

    def __str__(self):
        return f'{self.account_id},{round(self.balance,2)},{self.interest}'

    def _withdraw_(self):

        '''prompt the user to choose amount to withdraw'''

        print('Amount: ',flush = True)
        _amount_()
        deducted_amount = valid_amount[-1]

        '''prevent user from withdrawing more than their balance '''

        self.balance -=  deducted_amount
        if self.balance < 0:
            print('Cannot proceed your withdrawal')
            self.balance += deducted_amount
    
class Checking(Account):

    '''inherit account class and force interest to be 0'''
    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 0
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id}  {round(float(self.balance),2)}'

class Savings(Account):

    '''inherit account class and force interest to be 1'''

    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 1
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id} {round(float(self.balance),2)}'

## Ivan: credit and customer class HERE, remember to convert balance or limit to float because originally they are string
    
## Ivan: for Credit class
def __credit_card_charge__(self):

    '''promt user to choose amount'''

    print('Amount: ', flush = True)
    _amount_()
    increased_amount = valid_amount[-1]

    '''prevent user to charge more than their limit'''
        
    self.balance += increased_amount
    if increased_amount > self.limit:
        print('You have passed your credit limit')
        self.balance -= increased_amount


In [12]:
'''for storing newly created Customer object'''
customer_collection = {}

valid_name = []
def name_lookup():

    '''prompt user to put in only valid customer name and account type'''

    try:
        account_name = input('Customer name:')
        account_type = input('Checking or Savings or Credit')
        if account_name not in customer_collection.keys() or account_type not in ['Savings','savings','checking','Checking','credit',"Credit"]:
            raise ValueError('invalid name/account type')
        else:
            valid_name.append((account_name,account_type))
    except ValueError:
        name_lookup()

def interface():

    ## Ghaith & Dallen: import function HERE

    '''display options and prompt user to choose'''

    print(['1. View Customer\n',
            '2. Deposit\n',
            '3. Withdraw\n',
            '4. Credit Card Charge\n',
            '5. Credit Card Payment\n',
            '6. Exit\n'])
    user_choice = input('Enter your 1/2/3/4/5/6')
    while user_choice not in str(list(range(1,8))):
        user_choice = input('Enter your 1/2/3/4/5/6')

    

    ## Ghaith & Dallen: View customer and deposit HERE 
    ## Change if user_choise == '3  to elif 
    '''withdraw action'''

    if user_choice == '3':

        '''prompt the user to choose customer name and account type'''
        '''avoid typo/invalid name/account'''
                    
        name_lookup()
        account_name,account_type = valid_name[-1]

            
        '''proceed withdrawing by calling withdraw method for corresponding account type'''

        if account_type in ['Checking','checking']:
            customer_collection[account_name].checking._withdraw_()
        elif account_type in ['Savings','savings']:
            customer_collection[account_name].savings._withdraw_()

        '''display customer account after withdrawing'''
            
        # print(customer_collection[account_name])
            
        '''creditcard charge method'''
            
    elif user_choice == '4':
            
        '''prompt user to choose customer name and proceed the method'''
        name_lookup()
        account_name,account_type = valid_name[-1]
            
        '''perform credit card charge action'''

        customer_collection[account_name].credit.__credit_card_charge__()

        # print(customer_collection[account_name])

    ## Ivan: Credit card payment and exit HERE
    ## ivan, he said that exit means write all the change that the user made in a csv file, so I think we will use 'w' for open command. Simply perform like view customer option but display it in a new csv file. Then terminate the function. 

['1. View Customer\n', '2. Deposit\n', '3. Withdraw\n', '4. Credit Card Charge\n', '5. Credit Card Payment\n', '6. Exit\n']


In [17]:
class Credit(Account):
    """
    inheriting class Account()
    interest of 30% also in __init__
    additional attribute -- "credit limit" -- max value allowed for balance
    """
    def __init__(self,account_id, balance, interest, credit_limit):
        super().__init__(account_id,balance,interest)
        self.interest = interest
        self.credit_limit = credit_limit

    def __str__(self):
        return f"{self.account_id} {round(float(self.balance),2)} credit limit - {self.credit_limit}"
    
    def __credit_card_charge__(self):

        '''promt user to choose amount'''

        print('Amount: ', flush = True)
        _amount_()
        increased_amount = valid_amount[-1]

        '''prevent user to charge more than their limit'''
            
        self.balance += increased_amount
        if increased_amount > self.limit:
            print('You have passed your credit limit')
            self.balance -= increased_amount

In [18]:
class Customer():
    def __init__(self,username,checking,savings,credit):
        if type(username) != str:
            raise TypeError(f"username : {username} not a string")
        self.username = username

        if isinstance(checking, Checking()) == False:
            raise TypeError("checking not a Checking() object")
        self.checking = checking

        if isinstance(savings,Savings()) == False:
            raise TypeError("savings not a Savings() Object")
        self.savings = savings
        
        if isinstance(credit, Credit()) == False:
            raise TypeError("credit not a Credit() object")
        self.credit = credit

In [ ]:
def credit_card_payment():
    """     
    select customer
    select checking or savings
    select dollar amount
    lowers credit card "balance"?
    loweres checkings or savings balance
    """

def exit_interface():
    """
    prompt for address of csv file wanted to be created
    put all date in csv file
    terminate everything basically
    """